# Completely Replacing Your Branch

What do you do if you want to get rid of your version of a branch,
and replace it with someone else's? 

Not just merge his changes into yours, mind you, but really, completely *replace* yours with his?

You can do that, and it's not hard. A demo follows.

## Setting up a playground

Begin by setting up a simple repository, with only a master branch.

In [ ]:
sandbox=/tmp/sandbox
rm -rf $sandbox   # cleanliness is next to godliness
mkdir $sandbox    # create a sandbox
cd $_
git init scratch  # create a scratch repo
cd $_             # and cd into it

In [ ]:
pwd                         # are we there yet?
cat > README.md << __EOF__  # put something in it
# Scratch
A scratch repo for demos. 
__EOF__

git add README.md           # and get it into git
git commit -m"Add a README"

In [ ]:
cat README.md

Next, let's add a branch and do a little work in it.

In [ ]:
git checkout -b davidian  # create a branch and check it out
mkdir performance-reviews

# one employee
echo "WoW (Walks on Water)" > performance-reviews/willllllllson
git add !$
git commit -m"Willllllllson PR"

# and a second
echo "Can't tell his ass from a hole in the ground." > performance-reviews/haemer
git add !$
git commit -m"Haemer PR"


Switch back to master and do someting there, too, to make it distinct from the branch.

In [ ]:
git checkout master
mkdir wisdom
echo "It's a beautiful day in the neighborhood." > wisdom/rogers
echo "Few things are harder to put up with than the annoyance of a good example." > wisdom/twain
echo "If you listen carefully, you get to hear everything you didn't want to hear in the first place" > wisdom/aleichem
echo "Always carry a knife with you, just in case there's cheesecake or you need to stab someone in the throat." > wisdom/mattis
git add wisdom
git commit -m"Words of wisdom."

Add a popular, convenient monitoring command and look back at what we've done.

In [ ]:
git config --global alias.lol 'log --all --decorate --oneline --graph'  # add a typing convenience
git lol                                                                 # and try it out

Finally, let's create a company-wide Git server on a dedicated machine, maintained by our highly-paid SREs.
Call that machine, oh ... *localhost* :-)
Once that's done, we can push our repository to it for general access.

In [ ]:
repos=$sandbox/git-server
mkdir $repos                                                              # a place for all our repos
git clone --bare $PWD $repos/scratch.git                                  # create a bare repo
git daemon --export-all --enable=receive-pack --base-path=$repos $repos & # serve with git protocol

Did that work? Let's check by cloning it.

In [ ]:
cd $sandbox.                                     # get back, Jo-Jo
rm -rf scratch                                   # no guts, no glory!
git clone git://localhost/scratch.git scratch.me # this should clone the repo from the server
cd scratch.me                                    # Once more unto the breach, dear friends, once more;
git remote -v

Finally, verify the work's all there.

In [ ]:
git lol

Note, though, that it's all there in the server (`origin`). Locally, we now have only `master`.
Let's also get `davidian`.

In [ ]:
git checkout davidian
git lol

## Replacing a Problem Branch

Concerned that young Haemer could be disheartened by your harsh assessment of his abilities, 
your boss, Mr. Washcloth, suggests you rephrase it.

Teaching by example, Mr. Washcloth stays late, clones the repository himself, removes your appraisal,
writes an all-new version, and replaces the old version on the server.

In [ ]:
cd ..                                                   # back up a level
rm -rf scratch.washcloth                                # a clean slate for Mr. Washcloth
git clone git://localhost/scratch.git scratch.washcloth # Mr. Washcloth clones the repo
cd $_                                                   # now in Mr. Washcloth's clone
git checkout davidian                                   # the branch with all the appraisals
git reset HEAD~                                         # back up one version
cat << __EOF__ > performance-reviews/haemer
Although a hard worker, Haemer still often encounts difficulties distinguishing between
the terminal end of his alimentary canal and a depression in the earth\'s surface.
__EOF__
git add performance-reviews/haemer                       # pop-quiz: why can't you just use `git commit -am` here --
git commit -m"managerially worded Haemer PR"             #           one step, instead of two?

In [ ]:
git status
git lol

Because Mr. Washcloth's `davidian` branch isn't a descendant of `origin/davidian`, he can't push it.
He could do a `git pull` and a `git merge`, but that would leave your original version in the history.
He doesn't want that, so he managerially stomps all over your version, rewriting history.

In [ ]:
git push --force
git status
git lol

Next, let's return to your local clone of the repo.
You come back the next morning and, as far as you can see, nothing has changed.

In [ ]:
cd $sandbox/scratch.me  # go back to your version
git status
git lol

A `git fetch` shows you the problem.

In [ ]:
git fetch # get all the remote objects
git lol

Could you pull your manager's version and merge it onto the top of yours?
Sure, but your original would still be reachable.
Plus, you couldn't just push the result, because now the *server*'s branch wouldn't be a descendent of yours.

This, by the way, illustrates why we say, "Don't ever rebase something that's already public."
Changing history rarely works out well in either science fiction or in Git.

Rules are made to be broken. If you're truly satisfied with Mr. Washcloth's version of this branch,
and never want to see yours again, just replace your version with his, like this:

In [ ]:
git reset --hard origin/davidian # completely replace your `davidian` branch with `origin/davidian`
git lol
git status

## *Aaaaand ... You're Done!*